In [10]:
import numpy as np
import pandas as pd
import pickle as pkl
import xarray as xr
import copy
import os
import sys
sys.path.append(os.path.realpath('../split-data/'))
import return_period_tools as tools
import metrics
import random

import matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [11]:
# Convert flow to   CFS mm -> ft     km^2 -> ft^2    hr->s
conversion_factor = 0.00328084 * 10763910.41671 / 3600 / 24

In [12]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

In [13]:
# Camels attributes with RI information
dataName = '../data/camels_attributes.csv'
# load the data with pandas
pd_attributes = pd.read_csv(dataName, sep=',', index_col='gauge_id')

# Add the basin ID as a 8 element string with a leading zero if neccessary
basin_id_str = []
for a in pd_attributes.index.values:
    basin_id_str.append(str(a).zfill(8))
pd_attributes['basin_id_str'] = basin_id_str

In [14]:
def calculate_all_metrics_for_frequency_analysis(analysis_dict, flows):
    
    sims = list(flows.keys())[:-1]

    for metric in metrics.get_available_metrics():

        score = {sim:0 for sim in sims}
        
        if metric == 'NSE':
            for sim in sims:
                score[sim] = metrics.nse(flows['obs'],flows[sim])
        if metric == 'MSE':
            for sim in sims:
                score[sim] = metrics.mse(flows['obs'],flows[sim])
        if metric == 'RMSE':
            for sim in sims:
                 score[sim] = metrics.rmse(flows['obs'],flows[sim])
        if metric == 'KGE':
            for sim in sims:
                score[sim] = metrics.kge(flows['obs'],flows[sim])
        if metric == 'Alpha-NSE':
            for sim in sims:
                score[sim] = metrics.alpha_nse(flows['obs'],flows[sim])
        if metric == 'Beta-NSE':
            for sim in sims:
                score[sim] = metrics.beta_nse(flows['obs'],flows[sim])
        if metric == 'Pearson-r':
            for sim in sims:
                score[sim] = metrics.pearsonr(flows['obs'],flows[sim])
        if metric == 'Peak-Timing':
            for sim in sims:
                score[sim] = np.abs(metrics.mean_peak_timing(flows['obs'],flows[sim]))
        if metric == 'FHV':
            for sim in sims:
                score[sim] = metrics.fdc_fhv(flows['obs'],flows[sim])          
        if metric == 'FLV':
            for sim in sims:
                score[sim] = metrics.fdc_flv(flows['obs'],flows[sim])          
        if metric == 'FMS':
            for sim in sims:
                score[sim] = metrics.fdc_fms(flows['obs'],flows[sim])          
          
        for sim in sims:
            analysis_dict[metric][sim].append(score[sim])
        
    return

# ---------------------------------
# Return period split

The third test period (based on return periods) allows us to benchmark only on water years that contain streamflow events thatare larger (per basin)  than anything seen in the training data (<= 5-year return periods in training  and > 5-year return periods intesting). Model performances generally improve overall in this period according to the three correlation-based metrics (NSE,KGE, Pearson-r), but degrade according to the variance-based metric (alpha-NSE). This is expected due to the nature of the295metrics themselves – hydrology models generally show better correlation with observation in wet conditions simply due tohigher variability. However, the data-driven models remained better than both benchmark models against all four of thesemetrics. The bias metric (beta-NSE) was less consistent across test periods, however the data-driven models had less overallbias than both benchmark models in the return-period test period


# Return period split
# ---------------------------------

In [15]:
#-------------------------------------------------------------------------------------------------
# Set up lists to use in loops
models =        ['lstm','mc', 'sac', 'nwm']
flows =         ['lstm','mc', 'sac', 'nwm', 'obs']
#-------------------------------------------------------------------------------------------------

In [19]:
with open('./model_output_for_analysis/nwm_chrt_v2_1d_local.p', 'rb') as fb:
    nwm_results = pkl.load(fb)

with open('./model_output_for_analysis/lstm_return_period_split.p', 'rb') as fb:
    lstm_results_return_period_split = pkl.load(fb)
with open('./model_output_for_analysis/mclstm_return_period_split.p', 'rb') as fb:
    mclstm_results_return_period_split = pkl.load(fb)
with open('./model_output_for_analysis/sacsma_return_period_split.p', 'rb') as fb:
    sacsma_results_return_period_split = pkl.load(fb)

with open('../split-data/per_basin_train_periods_file.plk', 'rb') as fb:
    hp_train_periods = pkl.load(fb)
with open('../split-data/per_basin_val_periods_file.plk', 'rb') as fb:
    hp_val_periods = pkl.load(fb)
with open('../split-data/per_basin_test_periods_file.plk', 'rb') as fb:
    hp_test_periods = pkl.load(fb)
    
basin_list = sacsma_results_return_period_split.keys()

In [20]:
analysis_dict_all = {}
            

#-------------------------------------------------------------------------------------------------
#-----LOOP THROUGH BASINS------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------

for ib, basin_0str in enumerate(list(sacsma_results_return_period_split.keys())): 
    basin_int = int(basin_0str)

    #-------------------------------------------------------------------------------------------------
    # Get the NWM data for this basin in an xarray dataset.
    xr_nwm = xr.DataArray(nwm_results[basin_0str]['streamflow'].values, 
             coords=[nwm_results[basin_0str]['streamflow'].index], 
             dims=['datetime'])
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    # Get the HP training years for this basin.
    date_range_list = []
    for start_test_period, end_test_period in zip(hp_test_periods[basin_0str]['start_dates'],
                                                    hp_test_periods[basin_0str]['end_dates']):
        startx_time = min(max(start_test_period, pd.to_datetime('1996-10')), pd.to_datetime('2014-09'))
        endx_time = max(min(end_test_period, pd.to_datetime('2014-09')),pd.to_datetime('1996-10')) 
        if endx_time > startx_time:
            date_range = pd.date_range(startx_time,endx_time)
            date_range_list.extend(date_range)
    for start_val_period, end_val_period in zip(hp_val_periods[basin_0str]['start_dates'],
                                                    hp_val_periods[basin_0str]['end_dates']):
        startx_time = min(max(start_val_period, pd.to_datetime('1996-10')), pd.to_datetime('2014-09'))
        endx_time = max(min(end_val_period, pd.to_datetime('2014-09')),pd.to_datetime('1996-10')) 
        date_range = pd.date_range(startx_time,endx_time)
        if endx_time > startx_time:
            date_range = pd.date_range(startx_time,endx_time)
            date_range_list.extend(date_range)
    if len(date_range_list) < 1:
        continue
    #-------------------------------------------------------------------------------------------------        

    #-------------------------------------------------------------------------------------------------
    # Setting up the dictionary for the single basin results. Then will add to the overall dict.
    analysis_dict = {metric:{model:[] for model in models} for metric in metrics.get_available_metrics()}
    #-------------------------------------------------------------------------------------------------



    #-------------------------------------------------------------------------------------------------
    # We need the basin area to convert to CFS, to interpolate the RI from LPIII
    basin_area = pd_attributes.loc[basin_int, 'area_geospa_fabric']
    basin_str = tools.gauge_id_str(basin_int)
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    # Make dictionary with all the flows
    flow_mm = {}
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------        
     # NWM data
    sim_nwm = xr_nwm.loc[date_range_list]
    # convert from CFS to mm/day
    # fm3/s * 3600 sec/hour * 24 hour/day / (m2 * mm/m)
    flow_mm['nwm'] = sim_nwm*3600*24/(basin_area*1000)
    #-------------------------------------------------------------------------------------------------
    # Standard LSTM data trained on all years
    xrr = lstm_results_return_period_split[basin_0str]['1D']['xr']['QObs(mm/d)_sim']
    flow_mm['lstm'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_range_list]
    #-------------------------------------------------------------------------------------------------        
    # Mass Conserving LSTM data
    xrr = mclstm_results_return_period_split[basin_0str]['1D']['xr']['QObs(mm/d)_sim']
    flow_mm['mc'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_range_list]
    #-------------------------------------------------------------------------------------------------
    # SACSMA Sinlge run
    df = sacsma_results_return_period_split[basin_0str]
    flow_mm['sac'] = df.loc[date_range_list]
    #-------------------------------------------------------------------------------------------------
    # OBSERVATIONS
    xrr = mclstm_results_return_period_split[basin_0str]['1D']['xr']['QObs(mm/d)_obs']
    flow_mm['obs'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_range_list]
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    # Make all xarray data similar
    for iflow in flows:
        if iflow == 'nwm': #already in the correct format
            continue
        if iflow == 'sac': #already in the correct format
            flow_mm[iflow] = xr.DataArray(np.array(flow_mm[iflow].values, dtype='float32'), 
                           coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
        else:
            flow_mm[iflow] = xr.DataArray(flow_mm[iflow].values[:,0], 
                           coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    calculate_all_metrics_for_frequency_analysis(analysis_dict, flow_mm)
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    #Now that the basin has been analyzed successfully, add it to the larger dictionary
    analysis_dict_all[basin_0str] = analysis_dict
    #------------------------------------------------------------------------------------------------- 

    
analysis_dict = {metric:{model:[] for model in models} for metric in metrics.get_available_metrics()}
for ib, basin_0str in enumerate(sacsma_results_return_period_split.keys()):
    try:
        for metric in metrics.get_available_metrics():
            for model in models:
                analysis_dict[metric][model].extend(analysis_dict_all[basin_0str][metric][model])
    except:
        continue
        
table_metrics = ['NSE','KGE','Pearson-r','Alpha-NSE','Beta-NSE','FHV','FLV','FMS']
round_to = 2
for metric in table_metrics:
    print('{} (median), {}, {}, {}, {}'.format(metric,
                                          np.round(np.nanmedian(analysis_dict[metric]['lstm']),round_to),
                                          np.round(np.nanmedian(analysis_dict[metric]['mc']),round_to),
                                          np.round(np.nanmedian(analysis_dict[metric]['sac']),round_to),
                                          np.round(np.nanmedian(analysis_dict[metric]['nwm']),round_to)))
print('Peak-Timing (mean), {}, {}, {}, {}'.format(np.round(np.nanmean(analysis_dict['Peak-Timing']['lstm']),round_to),
                                          np.round(np.nanmean(analysis_dict['Peak-Timing']['mc']),round_to),
                                          np.round(np.nanmean(analysis_dict['Peak-Timing']['sac']),round_to),
                                          np.round(np.nanmean(analysis_dict['Peak-Timing']['nwm']),round_to)))

/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning

NSE (median), 0.82, 0.78, 0.64, 0.67
KGE (median), 0.79, 0.74, 0.56, 0.64
Pearson-r (median), 0.92, 0.9, 0.84, 0.85
Alpha-NSE (median), 0.84, 0.79, 0.71, 0.79
Beta-NSE (median), -0.01, -0.02, 0.05, -0.04
FHV (median), -15.62, -19.72, -27.51, -20.42
FLV (median), 11.74, -36.34, 49.95, 10.81
FMS (median), -7.59, -6.57, -30.64, -4.31
Peak-Timing (mean), 0.36, 0.38, 0.74, 0.62


# ---------------------------------
# Standard Time Split

test period(1989-1999) is the same period used by previous studies  which allows us to confirm that the deep learning models (LSTM andMC-LSTM) trained for this project perform as expected relative to prior work. 
These metrics are broadly equivalent to thosereported for single models (not ensembles) by (Kratzert et al., 2019c) (LSTM) and (Hoedt et al., 2021) (MC-LSTM)

# Standard Time Split
# ---------------------------------

In [9]:
#-------------------------------------------------------------------------------------------------
# Set up lists to use in loops
models =        ['lstm','mc', 'sac']
flows =         ['lstm','mc', 'sac', 'obs']
#-------------------------------------------------------------------------------------------------

with open('./model_output_for_analysis/lstm_time_split1.p', 'rb') as fb:
    lstm_results_time_split1 = pkl.load(fb)
with open('./model_output_for_analysis/mclstm_time_split1.p', 'rb') as fb:
    mclstm_results_time_split1 = pkl.load(fb)
with open('./model_output_for_analysis/sacsma_time_split1.p', 'rb') as fb:
    sacsma_results_time_split1 = pkl.load(fb)
    
basin_list = sacsma_results_return_period_split.keys()

analysis_dict_all = {}
#-------------------------------------------------------------------------------------------------            

#-------------------------------------------------------------------------------------------------
#-----LOOP THROUGH BASINS------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------

for ib, basin_0str in enumerate(basin_list): 
    basin_int = int(basin_0str)



    #-------------------------------------------------------------------------------------------------
    # Setting up the dictionary for the single basin results. Then will add to the overall dict.
    analysis_dict = {metric:{model:[] for model in models} for metric in metrics.get_available_metrics()}
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    # We need the basin area to convert to CFS, to interpolate the RI from LPIII
    basin_area = pd_attributes.loc[basin_int, 'area_geospa_fabric']
    basin_str = tools.gauge_id_str(basin_int)
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    #----  Set the time period for metrics   ---------------------------------------------------------
    date_from = '1989-10'
    date_to = '1999-09'
    #-------------------------------------------------------------------------------------------------    

    #-------------------------------------------------------------------------------------------------
    # Make dictionary with all the flows
    flow_mm = {}
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    # Standard LSTM data trained on all years
    xrr = lstm_results_time_split1[basin_0str]['1D']['xr']['QObs(mm/d)_sim']
    flow_mm['lstm'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
    #-------------------------------------------------------------------------------------------------        
    # Mass Conserving LSTM data
    xrr = mclstm_results_time_split1[basin_0str]['1D']['xr']['QObs(mm/d)_sim']
    flow_mm['mc'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
    #-------------------------------------------------------------------------------------------------
    # SACSMA Sinlge run
    df = sacsma_results_time_split1[basin_0str]
    flow_mm['sac'] = df.loc[date_from:date_to]
    #-------------------------------------------------------------------------------------------------
    # OBSERVATIONS
    xrr = mclstm_results_time_split1[basin_0str]['1D']['xr']['QObs(mm/d)_obs']
    flow_mm['obs'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    # Make all xarray data similar
    for iflow in flows:
        if iflow == 'nwm': #already in the correct format
            continue
        if iflow == 'sac': #already in the correct format
            flow_mm[iflow] = xr.DataArray(np.array(flow_mm[iflow].values, dtype='float32'), 
                           coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
        else:
            flow_mm[iflow] = xr.DataArray(flow_mm[iflow].values[:,0], 
                           coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    calculate_all_metrics_for_frequency_analysis(analysis_dict, flow_mm)
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    #Now that the basin has been analyzed successfully, add it to the larger dictionary
    analysis_dict_all[basin_0str] = analysis_dict
    #------------------------------------------------------------------------------------------------- 
    
analysis_dict = {metric:{model:[] for model in models} for metric in metrics.get_available_metrics()}
for ib, basin_0str in enumerate(basin_list):
    try:
        for metric in metrics.get_available_metrics():
            for model in models:
                analysis_dict[metric][model].extend(analysis_dict_all[basin_0str][metric][model])
    except:
        continue
        
table_metrics = ['NSE','KGE','Pearson-r','Alpha-NSE','Beta-NSE','FHV','FLV','FMS']
for metric in table_metrics:
    print('{} (median), {}, {}, {}'.format(metric,
                                          np.round(np.nanmedian(analysis_dict[metric]['lstm']),4),
                                          np.round(np.nanmedian(analysis_dict[metric]['mc']),4),
                                          np.round(np.nanmedian(analysis_dict[metric]['sac']),4)))
print('Peak-Timing (mean), {}, {}, {}'.format(np.round(np.nanmean(analysis_dict['Peak-Timing']['lstm']),4),
                                          np.round(np.nanmean(analysis_dict['Peak-Timing']['mc']),4),
                                          np.round(np.nanmean(analysis_dict['Peak-Timing']['sac']),4)))

/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning

NSE (median), 0.7491, 0.7447, 0.6211
KGE (median), 0.7586, 0.7573, 0.6091
Pearson-r (median), 0.8793, 0.8757, 0.8196
Alpha-NSE (median), 0.8663, 0.8469, 0.7932
Beta-NSE (median), -0.0366, -0.0293, 0.0651
FHV (median), -13.878, -15.0673, -19.0256
FLV (median), 13.2698, -32.5691, 48.8138
FMS (median), -3.7037, -7.633, -38.1623
Peak-Timing (mean), 0.4243, 0.4435, 0.8005


# ---------------------------------
# NWM Time Split

The second test period (1995-2014) allows us to benchmark against the NWM-Rv2,  which does not provide data prior to2851995. Most of these scores are broadly  equivalent to the metrics for the same models reported for the test period 1989-1999, with the exception of the FHV (high flow bias), FLV (low flow bias), add FMS (flow duration curve bias).  These metrics dependheavily on the observed flow characteristics during a particular test period and,  because they are less stable, are somewhat lessuseful in terms of drawing general conclusions.  We report them here primarily for continuity with previous studies (Kratzertet al., 2019c, b, 2020;  Frame et al., 2020; Nearing et al., 2020b; Klotz et al., 2021; Gauch et al., 2021a), and one of the  objectives of this paper (Section 2.4 is to expand on the high flow (FHV) analysis specifically

# NWM Time Split
# ---------------------------------

In [10]:
#-------------------------------------------------------------------------------------------------
# Set up lists to use in loops
models =        ['nwm', 'lstm','mc', 'sac']
flows =         ['nwm', 'lstm','mc', 'sac', 'obs']
#-------------------------------------------------------------------------------------------------

with open('./model_output_for_analysis/lstm_time_split2.p', 'rb') as fb:
    lstm_results_time_split2 = pkl.load(fb)
with open('./model_output_for_analysis/mclstm_time_split2.p', 'rb') as fb:
    mclstm_results_time_split2 = pkl.load(fb)
with open('./model_output_for_analysis/sacsma_time_split2.p', 'rb') as fb:
    sacsma_results_time_split2 = pkl.load(fb)
    
basin_list = sacsma_results_return_period_split.keys()

analysis_dict_all = {}
#-------------------------------------------------------------------------------------------------            

#-------------------------------------------------------------------------------------------------
#-----LOOP THROUGH BASINS------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------

for ib, basin_0str in enumerate(basin_list): 
    basin_int = int(basin_0str)


    #-------------------------------------------------------------------------------------------------
    # Get the NWM data for this basin in an xarray dataset.
    xr_nwm = xr.DataArray(nwm_results[basin_0str]['streamflow'].values, 
             coords=[nwm_results[basin_0str]['streamflow'].index], 
             dims=['datetime'])
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    # Setting up the dictionary for the single basin results. Then will add to the overall dict.
    analysis_dict = {metric:{model:[] for model in models} for metric in metrics.get_available_metrics()}
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    # We need the basin area to convert to CFS, to interpolate the RI from LPIII
    basin_area = pd_attributes.loc[basin_int, 'area_geospa_fabric']
    basin_str = tools.gauge_id_str(basin_int)
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    #----  Set the time period for metrics   ---------------------------------------------------------
    date_from = '1996-10'
    date_to = '2014-09'
    #-------------------------------------------------------------------------------------------------    

    #-------------------------------------------------------------------------------------------------
    # Make dictionary with all the flows
    flow_mm = {}
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------        
     # NWM data
    sim_nwm = xr_nwm.loc[date_from:date_to]
    # convert from CFS to mm/day
    # fm3/s * 3600 sec/hour * 24 hour/day / (m2 * mm/m)
    flow_mm['nwm'] = sim_nwm*3600*24/(basin_area*1000)
    #-------------------------------------------------------------------------------------------------
    # Standard LSTM data trained on all years
    xrr = lstm_results_time_split2[basin_0str]['1D']['xr']['QObs(mm/d)_sim']
    flow_mm['lstm'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
    #-------------------------------------------------------------------------------------------------        
    # Mass Conserving LSTM data
    xrr = mclstm_results_time_split2[basin_0str]['1D']['xr']['QObs(mm/d)_sim']
    flow_mm['mc'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
    #-------------------------------------------------------------------------------------------------
    # SACSMA Sinlge run
    df = sacsma_results_time_split2[basin_0str]
    flow_mm['sac'] = df.loc[date_from:date_to]
    #-------------------------------------------------------------------------------------------------
    # OBSERVATIONS
    xrr = mclstm_results_time_split2[basin_0str]['1D']['xr']['QObs(mm/d)_obs']
    flow_mm['obs'] = pd.DataFrame(data=xrr.values,index=xrr.date.values).loc[date_from:date_to]
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    # Make all xarray data similar
    for iflow in flows:
        if iflow == 'nwm': #already in the correct format
            continue
        if iflow == 'sac': #already in the correct format
            flow_mm[iflow] = xr.DataArray(np.array(flow_mm[iflow].values, dtype='float32'), 
                           coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
        else:
            flow_mm[iflow] = xr.DataArray(flow_mm[iflow].values[:,0], 
                           coords=dict(datetime=flow_mm[iflow].index.values), dims=['datetime'])
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    calculate_all_metrics_for_frequency_analysis(analysis_dict, flow_mm)
    #-------------------------------------------------------------------------------------------------


    #-------------------------------------------------------------------------------------------------
    #Now that the basin has been analyzed successfully, add it to the larger dictionary
    analysis_dict_all[basin_0str] = analysis_dict
    #------------------------------------------------------------------------------------------------- 
    
analysis_dict = {metric:{model:[] for model in models} for metric in metrics.get_available_metrics()}
for ib, basin_0str in enumerate(basin_list):
    try:
        for metric in metrics.get_available_metrics():
            for model in models:
                analysis_dict[metric][model].extend(analysis_dict_all[basin_0str][metric][model])
    except:
        continue
        
table_metrics = ['NSE','KGE','Pearson-r','Alpha-NSE','Beta-NSE','FHV','FLV','FMS']
for metric in table_metrics:
    print('{} (median), {}, {}, {}, {}'.format(metric,
                                          np.round(np.nanmedian(analysis_dict[metric]['lstm']),4),
                                          np.round(np.nanmedian(analysis_dict[metric]['mc']),4),
                                          np.round(np.nanmedian(analysis_dict[metric]['sac']),4),
                                          np.round(np.nanmedian(analysis_dict[metric]['nwm']),4)))
print('Peak-Timing (mean), {}, {}, {}, {}'.format(np.round(np.nanmean(analysis_dict['NSE']['lstm']),4),
                                          np.round(np.nanmean(analysis_dict['Peak-Timing']['mc']),4),
                                          np.round(np.nanmean(analysis_dict['Peak-Timing']['sac']),4),
                                          np.round(np.nanmean(analysis_dict['Peak-Timing']['nwm']),4)))

/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/jmframe/programs/anaconda3/envs/mc-paper/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning

NSE (median), 0.7411, 0.74, 0.5956, 0.6329
KGE (median), 0.7845, 0.7677, 0.5846, 0.6711
Pearson-r (median), 0.8778, 0.8783, 0.8119, 0.8222
Alpha-NSE (median), 0.9655, 0.9159, 0.8856, 0.8472
Beta-NSE (median), 0.0342, 0.026, 0.1275, -0.0171
FHV (median), -2.1456, -7.1599, -9.011, -13.8572
FLV (median), -14.0315, -26.8686, 39.9245, 2.6076
FMS (median), -9.2581, -11.5617, -28.8151, -5.5153
Peak-Timing (mean), 0.6795, 0.4619, 0.7716, 0.7191
